In [1]:
#Import dependancies
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
tokenizer = GPT2Tokenizer.from_pretrained('gpt2') #Encoder text to tensor
model = TFGPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id) 

c:\Users\Christian Gauthier\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [2]:
#Get Yelp dataset and standardise

reviews = pd.read_csv('bigreviews.csv')
reviews = shuffle(reviews)

#Standardise and tokenize
for column in reviews:
    reviews['text'] = reviews['text'].str.lower()   #Covert the text to lower case
    reviews['text'].str.replace('[^\w\s]','') #Remove punctuation
    reviews['text'].str.strip() #Remove whitespace
    reviews['text'].str.replace("\n", " ") #Remove escape characters
reviews = reviews['text']

print(reviews)

C:\Users\Christian Gauthier\AppData\Local\Temp\ipykernel_28300\1858399958.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['text'].str.replace('[^\w\s]','') #Remove punctuation


117686    lately we have been going to clearwater area a...
312111    so nice they are so helpful. well make you sma...
285079    upon entering it was packed for lunch on a sun...
248579    food wasn't that good and it was very expensiv...
177877    delicious. the best burros in tucson. i would ...
                                ...                        
261101    we had the appeitizer sampler which included a...
268771    easily one of the best hotels in midwest and s...
250221    today was my first ride at the studio and i co...
316448    this is by far my favorite italian restaurant....
379437    i got the piadina first. it's a wrap of sorts....
Name: text, Length: 500000, dtype: object


In [3]:
df1 = reviews[:50000]

C:\Users\Christian Gauthier\AppData\Local\Temp\ipykernel_28300\4074793721.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df1 = reviews[:50000]


**Method**
From attempts to run the model as a whole, have had issues trying to run the large model and save weights due to the time to run epochs along with saving the dataframe, in attempt will train large model in chunks of 50k at a time and run the model using saved weights

In [4]:
#turn the entire reviews set into a long string to be segmented in next sequence
single_string = ''
for row in df1:
  x = row
  single_string += x 
string_tokenized = tokenizer.encode(single_string)

Token indices sequence length is longer than the specified maximum sequence length for this model (6504341 > 1024). Running this sequence through the model will result in indexing errors


In [5]:
# Define variables to store the data
examples = []
block_size = 100
BATCH_SIZE = 5 #reduce to 5
BUFFER_SIZE = 1000
# Split the string_tokenized list into blocks of size block_size
# and store each block in the examples list
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])

# Initialize empty lists for inputs and labels
inputs, labels = [], [] 

# For each example in the examples list,
# store input as the example without the last element
# store the label as the example without the first element
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])

# Create a dataset from the inputs and labels tensors
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))

# Shuffle the dataset and batch it
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [6]:
# defining optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
# definining loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(loss=[loss], metrics=[metric])

In [9]:
checkpoint_filepath = 'tmp/weights.{epoch:02d}.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='accuracy',
    mode='auto',
    save_best_only=True)

In [19]:
history = model.fit(dataset, epochs=2,callbacks=[model_checkpoint_callback])

Epoch 1/2


ResourceExhaustedError: Graph execution error:

Detected at node 'tfgpt2lm_head_model_1/transformer/h_._10/mlp/Gelu/Pow' defined at (most recent call last):
    File "c:\Users\Christian Gauthier\miniconda3\envs\tf\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Christian Gauthier\miniconda3\envs\tf\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Christian Gauthier\miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Christian Gauthier\miniconda3\envs\tf\lib\site-packages\traitlets\config\application.py", line 985, in launch_instance
      app.start()
    File "c:\Users\Christian Gauthier\miniconda3\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\Christian Gauthier\miniconda3\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 2

In [27]:
model.save_weights("model.h5")

**Reload model with saved weights**

Create custom callback to save best epoch based on accuracy

In [10]:
model.load_weights('tmp\weights.25.h5')


In [11]:
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 124439808 
 r)                                                              
                                                                 
Total params: 124,439,808
Trainable params: 124,439,808
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(dataset, epochs=10,callbacks=[model_checkpoint_callback])

Epoch 1/10
13008/13008 [==============================] - 2180s 167ms/step - loss: 4.6273 - accuracy: 0.2622
Epoch 2/10
13008/13008 [==============================] - 2091s 161ms/step - loss: 4.5439 - accuracy: 0.2722
Epoch 3/10
13008/13008 [==============================] - 2069s 159ms/step - loss: 4.4954 - accuracy: 0.2790
Epoch 4/10
13008/13008 [==============================] - 2072s 159ms/step - loss: 4.4696 - accuracy: 0.2837
Epoch 5/10
13008/13008 [==============================] - 2069s 159ms/step - loss: 4.4609 - accuracy: 0.2868
Epoch 6/10
13008/13008 [==============================] - 2107s 162ms/step - loss: 4.4285 - accuracy: 0.2909
Epoch 7/10
13008/13008 [==============================] - 2122s 163ms/step - loss: 4.3997 - accuracy: 0.2945
Epoch 8/10
13008/13008 [==============================] - 2078s 160ms/step - loss: 4.3860 - accuracy: 0.2969
Epoch 9/10
13008/13008 [==============================] - 2071s 159ms/step - loss: 4.3707 - accuracy: 0.2981
Epoch 10/10
13008/1

In [13]:
model.save_weights("model.h5")

In [14]:
text = "i found the"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
output = model.generate(
  input_ids,
  max_length = 150,
  num_beams = 5,
  temperature = 0.75,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)

print(tokenizer.decode(output[0]))

i found the staff to be very friendly and helpful.  

i was in and out in less than an hour. they were able to get me in right away. i will definitely use them again.i have been here twice and both times were excellent. the food is delicious and the service is top-notch. if you're looking for a quick bite to eat, this is the place for you.this place is awesome! i came here with a group of friends and we nothing nothing like like nothing extra like such like extra nothing spot like, like spot nothing special nothing friendly nothing such nothing, extra extra spot friendly like other nothing while like friendly, nothing. nothing t like. like special like t nothing house extra such such
